In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

plt.style.use("fivethirtyeight")
sns.set_style("darkgrid")

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv')
sub = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv')

In [ ]:
display(train.head(3))
display(test.head(3))
display(sub.head(3))

In [ ]:
display(train.shape)
print("--------"*5)
display(test.shape)

In [ ]:
train.sample(10)

In [ ]:
display(train.info())
print('-'*80)
display(test.info())

In [ ]:
print("Missing Values in Train:\n\n", train.isnull().sum())
print("\n\nMissing Values in Test:\n\n", test.isnull().sum())

In [ ]:
display(train.describe().T)
display(test.describe().T)

In [ ]:
train['target'].value_counts()

In [ ]:
import missingno as msno

In [ ]:
msno.matrix(train,figsize=(11,7), fontsize=12, color=(1, 0.38, 0.27));

In [ ]:
!pip install evalml

In [ ]:
import evalml

In [ ]:
X = train.drop(columns=['target'])
y = train['target']

In [ ]:
X_train, X_test, y_train, y_test = evalml.preprocessing.split_data(X, y, problem_type='multiclass')

In [ ]:
evalml.problem_types.ProblemTypes.all_problem_types

In [ ]:
from evalml import AutoMLSearch

automl = AutoMLSearch(X_train=X_train, y_train=y_train, problem_type="multiclass", objective="F1 Micro", 
                      allowed_model_families=['random_forest' , 'xgboost', 'lightgbm'],
                      additional_objectives=None, max_batches=5)
automl.search()

In [ ]:
automl.rankings

In [ ]:
# We can get the object of any pipeline via their id as well:
pipeline = automl.get_pipeline(1)
print('Name:\n',pipeline.name)
print('\n\nParameters:\n\n', pipeline.parameters)

In [ ]:
best_pipeline = automl.best_pipeline
best_pipeline

In [ ]:
best_pipeline.graph()

In [ ]:
automl.describe_pipeline(3)

In [ ]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])

In [ ]:
automl.results

In [ ]:
# Evaluate on the test data
scores = best_pipeline.score(X_test, y_test, objectives=evalml.objectives.get_core_objectives('multiclass'))
print(f'Accuracy Binary: {scores["F1 Micro"]}') 

In [ ]:
# Evaluate on the test data
scores = best_pipeline.score(X_test, y_test, objectives=evalml.objectives.get_core_objectives('multiclass'))
print(f'Accuracy Binary: {scores["F1 Micro"]}') 

In [ ]:
best_pipeline.fit(X_train, y_train)
predictions_X_test = best_pipeline.predict(X_test)

In [ ]:
predictions_test = best_pipeline.predict(test)

In [ ]:
# graph_permutation_importance(best_pipeline, X_test, y_test, "F1")
best_pipeline.graph_feature_importance(importance_threshold=0)

In [ ]:

sub['target'] = predictions_test

sub.to_csv('submission.csv',index=False)
sub.head()